# MathDoku Solver

Similar to the puzzles published under the KenKen trademark.

## Exploratory Work

First question...given a target number, arithmetic operator, and range of digits, can we show all combinations that produce the right target?

Starting with addition and multiplication as they're associative and commutative.


In [10]:
max_digit = 4
all_digits = range(1, max_digit+1)

In [45]:
target = 7
operation = sum
num_digits = 3

In [46]:
# First, n choose k (e.g. 6 digits, choose 3)
import itertools
[x for x in itertools.combinations(all_digits, num_digits)]

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

In [47]:
# Filter to just those that hit target
[x for x in itertools.combinations(all_digits, num_digits) if operation(x) == target]

[(1, 2, 4)]

In [48]:
import numpy
target = 24
operation = numpy.prod
[x for x in itertools.combinations(all_digits, num_digits) if operation(x)==target]

[(2, 3, 4)]

## First Complication

So far so good but there's a complication: if we use the rules from KenKen then digits can be repeated in a cage (as long as they're not also repeated in a row or column).


In [50]:
target = 16
[x for x in itertools.combinations(all_digits, num_digits) if operation(x)==target]

[]

In [51]:
2*2*4

16

It's OK though, because `itertools` has that covered in `combinations_with_replacement`.

In [52]:
[x for x in itertools.combinations_with_replacement(all_digits, num_digits) if operation(x)==target]

[(1, 4, 4), (2, 2, 4)]

## Second Complication

Next problem is that subtraction and division are not associative -- order changes the result. KenKen rules aren't particularly fussed about this, you decide the order in your head and that's fine. Generally the puzzles will only have 2 cells when the target must be produced with subtraction or division but apparently not all puzzles will do this.

In [53]:
def subtract(numbers):
    ret = numbers[0]
    for i in range(1, len(numbers)):
        ret -= numbers[i]
    return ret

def divide(numbers):
    ret = numbers[0]
    for i in range(1, len(numbers)):
        ret = ret // i
    return ret

So what we need to do here is, for each possible combination of digits, produce all the possible orderings of those digits. Because subtraction and division are not associative we'll need to use `product` instead of `combinations_with_replacement`. KenKen never uses negative targets in cages but some other puzzles do.

In [60]:
num_digits = 2
target = 2
operation = subtract

[x for x in itertools.product(all_digits, repeat=num_digits)]

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [62]:
[x for x in itertools.product(all_digits, repeat=num_digits) if operation(x) == target]

[(3, 1), (4, 2)]

# Representing MathDoku puzzles

These puzzles are a little more complicated to represent. We need:

* Standard N x N grid (latin squre)
* A list of "cages" -- target values and operations
* A mapping that tells us which cells are in which cages


In [88]:
import puzzlegrid as pg

grid = pg.ConstraintPuzzle(max_digit)

In [89]:
cages = [(16, numpy.product, 3), (7, sum, 3), (2, subtract, 2), (12, numpy.product, 3), (2, divide, 2), (2, divide, 2), (4, sum, 1)]

In [90]:
cage_cells = [[0, 0, 1, 1],
              [2, 0, 1, 6],
              [2, 3, 4, 4],
              [3, 3, 5, 5]]

In [91]:
grid.set(1, 3, 4)

In [92]:
print(grid)

.......4........


So let's get the possible values for each cage

In [96]:
cage_values = []
for i, c in enumerate(cages):
    target, operation, num = c
    combos = []
    if operation == sum or operation == numpy.product:
        combos = [x for x in itertools.combinations_with_replacement(all_digits, num)]
    else:
        combos = [x for x in itertools.product(all_digits, repeat=num)]
    cage_values.append([x for x in combos if operation(x) == target])
    print(i, cage_values[i])

0 [(1, 4, 4), (2, 2, 4)]
1 [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
2 [(3, 1), (4, 2)]
3 [(1, 3, 4), (2, 2, 3)]
4 [(2, 1), (2, 2), (2, 3), (2, 4)]
5 [(2, 1), (2, 2), (2, 3), (2, 4)]
6 [(4,)]


We already have a value for cell 1, 3 (4). Let's see if that helps us with any of the other cells in that row or column.

In [94]:
for j in range(max_digit):
    print((1, j), grid.get_allowed_values(1, j), cage_values[cage_cells[1][j]])

(1, 0) {1, 2, 3} [(3, 1), (4, 2)]
(1, 1) {1, 2, 3} [(1, 4, 4), (2, 2, 4)]
(1, 2) {1, 2, 3} [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
(1, 3) {4} [(4,)]


In [97]:
for i in range(max_digit):
    print((i, 3), grid.get_allowed_values(i, 3), cage_values[cage_cells[i][3]])

(0, 3) {1, 2, 3} [(1, 2, 4), (1, 3, 3), (2, 2, 3)]
(1, 3) {4} [(4,)]
(2, 3) {1, 2, 3} [(2, 1), (2, 2), (2, 3), (2, 4)]
(3, 3) {1, 2, 3} [(2, 1), (2, 2), (2, 3), (2, 4)]


Hmmm. So, basically, each cell can take any number from 1-4. 

This isn't going to work.

Better approach will be backtracking and not worrying about constraint propogation just yet.